In [1]:
import pandas as pd

In [4]:
import pandas as pd
import re

# Örnek veri
data = pd.read_csv('movies_with_years.csv')

# DataFrame oluşturuluyor
df = pd.DataFrame(data, columns=["Title"])

# Fonksiyon: Veriyi işleme
def parse_movie_data(row):
    data = row["Title"]
    
    # Tırnaklar arasındaki verileri tek bir alan olarak almak
    pattern = r'("[^"]*")'
    matches = re.findall(pattern, data)

    # Tırnaklar arasındaki metinleri koruyarak virgüllerle ayırma işlemi
    data_without_quotes = re.sub(pattern, '###QUOTED###', data)
    columns = data_without_quotes.split(',')

    # Eğer kolon sayısı eksikse, eksik olanları None ile dolduruyoruz
    while len(columns) < 9:
        columns.append(None)

    # "###QUOTED###" olan yerleri tekrar tırnaklı veri ile değiştirmek
    for i in range(len(columns)):
        # Eğer veri None değilse, o zaman işlem yapıyoruz
        if columns[i] and '###QUOTED###' in columns[i]:
            columns[i] = matches.pop(0).strip('"')

    # Sonuçları anlamlı hale getirmek
    result = {
        "Title": columns[0],
        "Year": columns[1],
        "Genre": columns[2],
        "Director": columns[3],
        "Actors": columns[4],
        "BoxOffice": columns[5],
        "Runtime": columns[6],
        "Poster": columns[7],
        "Country": columns[8]
    }

    return result

# DataFrame'deki her satıra uygulama
parsed_data = df.apply(parse_movie_data, axis=1)

# Yeni bir DataFrame oluşturuluyor
parsed_df = pd.json_normalize(parsed_data)

# Sonuçları gösterme
print(parsed_df)


                                           Title  Year  \
0     Star Wars: Episode VII - The Force Awakens  2015   
1                                 Jurassic World  2015   
2                                      Furious 7  2015   
3                        Avengers: Age of Ultron  2015   
4                                        Minions  2015   
...                                          ...   ...   
1913                                 The Servant  2010   
1914                              Norwegian Wood  2010   
1915                             Head Over Heels  2010   
1916                                13 Assassins  2010   
1917                                  Green Zone  2010   

                             Genre                   Director  \
0        Action, Adventure, Sci-Fi                J.J. Abrams   
1        Action, Adventure, Sci-Fi            Colin Trevorrow   
2                 Action, Thriller                  James Wan   
3        Action, Adventure, Sci-Fi         

In [5]:
# veriyi csv kaydet
parsed_df.to_csv('parsed_movies.csv', index=False)